In [1]:
import os

import pandas as pd

DATA_DIR = "../data/public"

table_names = [f"main{i}" for i in range(1, 4)]
table_paths = [os.path.join(DATA_DIR, f"{table_name}.csv") for table_name in table_names]
tables = [pd.read_csv(table_path) for table_path in table_paths]
len(tables)

KeyboardInterrupt: 

In [ ]:
tables[0].info()

In [ ]:
tables[1].info()

In [ ]:
tables[2].info()

In [ ]:
from app.custom_dataframe import CustomDataFrame

df = CustomDataFrame()
df.head()

In [ ]:
result_df = pd.DataFrame(columns=["uid", "uid1", "uid2", "uid3"])

for table_num, table in enumerate(tables):
    table = table.iloc[:10]
    for index in range(len(table)):
        row = table.iloc[index]
        response = df.register_row(row)
        print(response)